In [1]:
import pandas as pd
import glob
import datetime
import scipy
from sklearn.cluster import MeanShift

In [4]:
l = [pd.read_csv(filename) for filename in sorted(glob.glob("output/centroids/*.csv"))]
dados = pd.concat(l, axis=0, sort=False)
dados['DATETIME'] =  pd.to_datetime(dados['DATETIME'], format='%Y-%m-%d %H:%M:%S')
dados = dados.set_index('DATETIME')
dados.index = dados.index.map(lambda x: x.replace(second=0))
dados.head()

,N_Cluster,ID_CLUS,LAT,LON,DIST,IND_X,IND_Y,T_RELATION,RAIN_FALL,DBz
DATETIME,,,,,,,,,,
2014-09-28 19:24:00,12,0,-4.209177,-60.496387,129.979844,61,92,SW,115.717804,56.024703
2014-09-28 19:24:00,12,1,-2.268807,-60.567352,116.431544,169,88,NW,57.591373,51.176019
2014-09-28 19:24:00,12,2,-3.561566,-58.605930,160.661355,97,197,SE,40.754768,48.773155
2014-09-28 19:24:00,12,3,-2.304838,-60.315643,99.995682,167,102,NW,61.735973,51.658913
2014-09-28 19:24:00,12,4,-2.771915,-60.441680,65.067066,141,95,NW,59.813438,51.439080


In [5]:
freqtime = pd.date_range(start=dados.index.min(), end=dados.index.max(), freq='12T')
t1 = freqtime[0]
t2 = freqtime[1]
t3 = freqtime[2]

In [6]:
time1 = dados.loc[t1]
time1.head()

,N_Cluster,ID_CLUS,LAT,LON,DIST,IND_X,IND_Y,T_RELATION,RAIN_FALL,DBz
DATETIME,,,,,,,,,,
2014-09-28 19:24:00,12,0,-4.209177,-60.496387,129.979844,61,92,SW,115.717804,56.024703
2014-09-28 19:24:00,12,1,-2.268807,-60.567352,116.431544,169,88,NW,57.591373,51.176019
2014-09-28 19:24:00,12,2,-3.561566,-58.605930,160.661355,97,197,SE,40.754768,48.773155
2014-09-28 19:24:00,12,3,-2.304838,-60.315643,99.995682,167,102,NW,61.735973,51.658913
2014-09-28 19:24:00,12,4,-2.771915,-60.441680,65.067066,141,95,NW,59.813438,51.439080


In [7]:
time2 = dados.loc[t2]
time2.head()

,N_Cluster,ID_CLUS,LAT,LON,DIST,IND_X,IND_Y,T_RELATION,RAIN_FALL,DBz
DATETIME,,,,,,,,,,
2014-09-28 19:36:00,11,0,-4.173204,-60.568420,130.147843,63,88,SW,112.079689,55.802731
2014-09-28 19:36:00,11,1,-2.016774,-61.232365,186.247937,183,51,NW,85.160400,53.894103
2014-09-28 19:36:00,11,2,-3.274401,-61.161720,130.752088,113,55,SW,54.635250,50.809867
2014-09-28 19:36:00,11,3,-2.304872,-60.153824,95.011285,167,111,NW,66.665298,52.192697
2014-09-28 19:36:00,11,4,-2.789865,-60.477665,66.994555,140,93,NW,31.207119,46.918359


In [36]:
mat = scipy.spatial.distance.cdist(time1[['IND_X','IND_Y']], 
                              time2[['IND_X','IND_Y']], metric='euclidean')

In [44]:
from sklearn.metrics.pairwise import euclidean_distances
df =pd.DataFrame(euclidean_distances(t1,t2))

df.index =  t1.index
df.columns = t2.index
df['min_distance'] = df.min(axis=1)
df['min_distance_id'] = df.idxmin(axis=1)

KDTree = scipy.spatial.cKDTree(t1)
distances, indexes = KDTree.query(t2, n_jobs=-1)
df

,0,1,2,3,4,5,6,7,8,9,10,min_distance,min_distance_id
0,4.472136,128.705089,63.820060,107.689368,79.006329,107.298649,110.724884,68.308125,66.483081,50.447993,12.369317,4.472136,0
1,106.000000,39.560081,65.000000,23.086793,29.427878,4.123106,128.701204,46.238512,122.065556,109.658561,96.254870,4.123106,5
2,114.179683,169.446157,142.898565,110.887330,112.538882,133.454112,2.236068,129.294238,48.010416,60.926185,104.785495,2.236068,6
3,104.938077,53.450912,71.589105,9.000000,28.460499,18.027756,116.017240,51.400389,112.871608,101.769347,94.260278,9.000000,3
4,78.313473,60.827625,48.826222,30.528675,2.236068,29.154759,109.384642,28.301943,95.671312,82.097503,68.000000,2.236068,4
5,80.230917,157.136883,117.643529,106.000000,97.185390,123.166554,36.400549,107.782188,12.041595,26.305893,72.111026,12.041595,8
6,122.800651,3.605551,68.007353,58.694122,56.586217,32.695565,163.975608,57.567352,152.839785,138.683092,115.520561,3.605551,1
7,107.935166,63.788714,79.931220,2.236068,34.409301,29.000000,107.935166,59.396970,108.747414,99.040396,96.690227,2.236068,3
8,21.095023,122.090131,64.202804,93.134312,67.268120,96.540147,93.407708,62.681736,52.239832,35.014283,11.045361,11.045361,10
9,38.118237,138.974818,85.146932,102.107786,81.024688,110.308658,77.388630,81.394103,32.140317,16.552945,31.780497,16.552945,9


In [45]:
indexes

array([ 0,  6, 11,  7,  4,  1,  2, 11,  5,  9,  8])

In [23]:
t1 = pd.DataFrame({'x':time1['IND_X'].ravel(),'y':time1['IND_Y'].ravel()})
t2 = pd.DataFrame({'x':time2['IND_X'].ravel(),'y':time2['IND_Y'].ravel()})